In [8]:
%load_ext autoreload
%autoreload 2

from util_0704 import *
from myalgorithm_0704_1 import algorithm

import statsmodels.api as sm
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- 실험해볼 내용들

1. 좌표간 거리 계산에 유클리드 거리 대신에 민코프스키 거리를 사용하면 어떨까?

## 기본 코드

In [7]:
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'

In [74]:
with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

### 기존 유클리드 거리를 이용한 단순선형회귀분석

In [75]:
X2 = []
y = []
for i in range(K):
    for j in range(i + 1, K):
        x1 = ALL_ORDERS[i].shop_lat
        y1 = ALL_ORDERS[i].shop_lon

        x2 = ALL_ORDERS[j].shop_lat
        y2 = ALL_ORDERS[j].shop_lon

        dist = DIST[i][j]

        diff1 = abs(x1 - x2)
        diff2 = abs(y1 - y2)

        X2.append((diff1 ** 2 + diff2 ** 2) ** (1/2) * 125950)
        y.append(dist)

X = pd.DataFrame({'X_value': X2})

X = sm.add_constant(X)
model = sm.OLS(y, X)
result = model.fit()

print(result.rsquared)

0.9931478352343887


### 민코프스키 거리를 이용한 단순선형회귀분석

In [76]:
w = 1.6

X2 = []
X4 = []
y = []
for i in range(K):
    for j in range(i + 1, K):
        x1 = ALL_ORDERS[i].shop_lat
        y1 = ALL_ORDERS[i].shop_lon

        x2 = ALL_ORDERS[j].shop_lat
        y2 = ALL_ORDERS[j].shop_lon

        dist = DIST[i][j]

        X4.append(get_minkowski_dist_by_coords(x1, y1, x2, y2, w))
        y.append(dist)

X = pd.DataFrame({'X_value': X4})

X = sm.add_constant(X)
model = sm.OLS(y, X)
result = model.fit()

print(f'{w}: {result.rsquared}')

1.6: 0.994766479718545


#### 가중치 그리드 탐색

In [68]:
max_rsquared = -1
max_rsquared_w = -1
for w in range(1500, 1700):
    w /= 1000
    X2 = []
    X4 = []
    y = []
    for i in range(K):
        for j in range(i + 1, K):
            x1 = ALL_ORDERS[i].shop_lat
            y1 = ALL_ORDERS[i].shop_lon

            x2 = ALL_ORDERS[j].shop_lat
            y2 = ALL_ORDERS[j].shop_lon

            dist = DIST[i][j]

            X4.append(get_minkowski_dist_by_coords(x1, y1, x2, y2, w))
            y.append(dist)

    X = pd.DataFrame({'X_value': X4})

    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    result = model.fit()

    # print(f'{w}: {result.rsquared}')

    if result.rsquared > max_rsquared:
        max_rsquared = result.rsquared
        max_rsquared_w = w

print(max_rsquared_w, max_rsquared)

1.611 0.9947686632169045


### 철화 제출본 확인

In [1]:
from util_0704_ch import *
from myalgorithm_0704_ch import algorithm

In [4]:
problem_file = '../alg_test_problems_20240429/TEST_K100_2.json'

timelimit = 60

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])

solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)